In [1]:
import dgl
import copy
import glob
import math
import torch
import torch.nn as nn
import numpy as np
from os import path
from pathlib import Path
from trainAndEval import train, evaluate
from dgl.data import DGLDataset
from TrainResults import TrainResults
from dgl.dataloading import GraphDataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from CustomGCN_OnlyNFeatSumMsg import CustomGCN_OnlyNFeatSumMsg
from CustomGCN_OnlyNFeatMeanMsg import CustomGCN_OnlyNFeatMeanMsg
from ToyDGLDataset_v2 import ToyDGLDataset_v2, GetNodeFeatureVectors, GetEdgeFeatureVectors, GetNeighborNodes, GetEdgeList

Using backend: pytorch


In [2]:
# training
def trainEpochs(model, device, dataloader, optimizer, loss_fn, batchsize, nEpochs):
    results = TrainResults()
    bestModel = None
    bestValAcc = 0.0

    for epoch in range(nEpochs):
        # train
        epochLoss = train(model, device, dataloader, optimizer, loss_fn, batchsize, results)

        # evaluate
        train_result = evaluate(model, device, train_dataloader)
        val_result = evaluate(model, device, val_dataloader)
        test_result = evaluate(model, device, test_dataloader)

        results.addEpochResult(epochLoss, train_result, val_result, test_result)

        if(epoch % 5 == 0):
            results.printLastResult()

        if results.best_val_acc > bestValAcc:
            bestValAcc = results.best_val_acc
            bestModel = copy.deepcopy(model)

    return results, bestModel

In [3]:
def getAllDatasetNames(datasetRootDir):
    files = glob.glob(datasetRootDir + '/*/*/*.json', recursive=True)
    #files = [x for x in files if "Toy2_v3_1" in x]
    files.sort()
    datasetDirectories = [path.dirname(file) for file in files]
    datasetnames = [path.normpath(dir).split(path.sep)[-1] for dir in datasetDirectories]
    return datasetDirectories, datasetnames

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

datasetRootDir = '/home/andrew/GNN_Sandbox/GraphToyDatasets_v3'
datasetDirs, datasetNames = getAllDatasetNames(datasetRootDir)

In [5]:
import time
now = time.time()

batchSize = 1024
print(f'Device: {device}')

for i in range(len(datasetDirs)):
    dataset = ToyDGLDataset_v2(datasetNames[i], datasetDirs[i])
    splitIndices = dataset.get_split_indices()

    train_sampler = SubsetRandomSampler(splitIndices['train'])
    val_sampler = SubsetRandomSampler(splitIndices['valid'])
    test_sampler = SubsetRandomSampler(splitIndices['test'])

    train_dataloader = GraphDataLoader(dataset, sampler=train_sampler, batch_size=batchSize, drop_last=False)
    val_dataloader = GraphDataLoader(dataset, sampler=val_sampler, batch_size=batchSize, drop_last=False)
    test_dataloader = GraphDataLoader(dataset, sampler=test_sampler, batch_size=batchSize, drop_last=False)
    
    # Create the model with given dimensions
    model = CustomGCN_OnlyNFeatSumMsg(dataset.dim_nfeats, 16, dataset.num_graph_classes).to(device)
    model.reset_parameters()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
    loss_fn = nn.CrossEntropyLoss()
    epochs = 30
    
    # train
    print(f'Beginning training on dataset {datasetNames[i]}')
    results, bestmodel = trainEpochs(model, device, train_dataloader, optimizer, loss_fn, batchSize, epochs)
    results.printBestResult()
    
    # save results
    outputFolder = path.join(datasetDirs[i], 'CustomGCN_OnlyNFeatSumMsg')
    Path(outputFolder).mkdir(parents=True, exist_ok=True)
    
    results.savePlots(outputFolder)
    results.dumpSummary(outputFolder)
    results.pickledump(outputFolder)
    
    # save the best model for inference. (when loading for inference -> model.eval()!! )
    # https://pytorch.org/tutorials/beginner/saving_loading_models.html#what-is-a-state-dict
    torch.save(bestmodel.state_dict(), path.join(outputFolder, 'model.pt'))
    
    print(f'------------------({i+1}/{len(datasetDirs)}) models trained------------------\n')


end = time.time()
elapsed = end - now
print(f'{elapsed} seconds elapsed')

Device: cuda
Done loading data from cached files.
Beginning training on dataset Toy0_v3_1_0NodeCountMeanDiff
Epoch: 0, Loss: 99.3839, Train: 0.499, Valid: 0.503, Test: 0.506, AUC: 0.495
Epoch: 5, Loss: 2.1026, Train: 0.498, Valid: 0.502, Test: 0.506, AUC: 0.495
Epoch: 10, Loss: 0.8940, Train: 0.502, Valid: 0.497, Test: 0.494, AUC: 0.495
Epoch: 15, Loss: 0.8214, Train: 0.503, Valid: 0.501, Test: 0.506, AUC: 0.495
Epoch: 20, Loss: 0.7093, Train: 0.504, Valid: 0.498, Test: 0.498, AUC: 0.495
Epoch: 25, Loss: 0.7190, Train: 0.502, Valid: 0.497, Test: 0.494, AUC: 0.495
Best epoch: 
Epoch: 26, Loss: 0.7218, Train: 0.500, Valid: 0.505, Test: 0.504, AUC: 0.500
------------------(1/27) models trained------------------

Done loading data from cached files.
Beginning training on dataset Toy0_v3_1_11NodeCountMeanDiff
Epoch: 0, Loss: 0.9097, Train: 0.939, Valid: 0.937, Test: 0.942, AUC: 1.000
Epoch: 5, Loss: 0.0414, Train: 1.000, Valid: 1.000, Test: 1.000, AUC: 1.000
Epoch: 10, Loss: 0.0164, Train: 

Epoch: 25, Loss: 0.5849, Train: 0.737, Valid: 0.738, Test: 0.737, AUC: 0.829
Best epoch: 
Epoch: 18, Loss: 0.6001, Train: 0.743, Valid: 0.745, Test: 0.747, AUC: 0.829
------------------(12/27) models trained------------------

Done loading data from cached files.
Beginning training on dataset Toy2_v3_0_1percentDiff
Epoch: 0, Loss: 15.0476, Train: 0.493, Valid: 0.488, Test: 0.497, AUC: 0.487
Epoch: 5, Loss: 1.9540, Train: 0.502, Valid: 0.505, Test: 0.509, AUC: 0.509
Epoch: 10, Loss: 1.0895, Train: 0.502, Valid: 0.497, Test: 0.494, AUC: 0.509
Epoch: 15, Loss: 1.1072, Train: 0.502, Valid: 0.497, Test: 0.494, AUC: 0.509
Epoch: 20, Loss: 3.2506, Train: 0.502, Valid: 0.497, Test: 0.494, AUC: 0.509
Epoch: 25, Loss: 0.9234, Train: 0.502, Valid: 0.497, Test: 0.494, AUC: 0.509
Best epoch: 
Epoch: 27, Loss: 0.9765, Train: 0.508, Valid: 0.510, Test: 0.501, AUC: 0.504
------------------(13/27) models trained------------------

Done loading data from cached files.
Beginning training on dataset Toy2_

Epoch: 10, Loss: 1.5915, Train: 0.931, Valid: 0.929, Test: 0.932, AUC: 0.993
Epoch: 15, Loss: 0.1250, Train: 0.976, Valid: 0.978, Test: 0.977, AUC: 0.996
Epoch: 20, Loss: 0.1246, Train: 0.975, Valid: 0.976, Test: 0.976, AUC: 0.996
Epoch: 25, Loss: 0.1052, Train: 0.975, Valid: 0.976, Test: 0.976, AUC: 0.996
Best epoch: 
Epoch: 26, Loss: 0.1028, Train: 0.976, Valid: 0.978, Test: 0.977, AUC: 0.996
------------------(24/27) models trained------------------

Done loading data from cached files.
Beginning training on dataset Toy2_v3_1_5NodeCountMeanDiff
Epoch: 0, Loss: 6.0060, Train: 0.593, Valid: 0.587, Test: 0.591, AUC: 0.836
Epoch: 5, Loss: 0.2353, Train: 0.976, Valid: 0.978, Test: 0.974, AUC: 0.996
Epoch: 10, Loss: 0.1418, Train: 0.988, Valid: 0.989, Test: 0.986, AUC: 0.999
Epoch: 15, Loss: 0.1027, Train: 0.988, Valid: 0.989, Test: 0.987, AUC: 0.999
Epoch: 20, Loss: 0.0778, Train: 0.988, Valid: 0.989, Test: 0.986, AUC: 0.999
Epoch: 25, Loss: 0.0633, Train: 0.988, Valid: 0.989, Test: 0.98